# **C.O.L.L.A.B.**
<hr style="border:2px solid black">

## Goal: 
* 
* 
<hr style="border:2px solid black">

# Imports

In [1]:
import pandas as pd
import numpy as np
from wrangle import wrangle
from sklearn.model_selection import train_test_split
import scipy.stats as stats
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from sklearn.model_selection import learning_curve

<hr style="border:2px solid black">

# **Acquire**
Plan --> **Acquire** --> Prepare --> Explore --> Model --> Deliver

* 

In [2]:
    # Acquire kaggle data
    col = pd.read_csv('cost_of_living_msa.csv')

    # Acquire Census Data
    family_income = pd.read_csv('B19126_msa_2022.csv')
    family_count = pd.read_csv('DP02_msa_2022.csv')

In [3]:
    # grab the indices 6 and 7 from the dataset and store as male
    male = family_count[6:8]
    # grab the indices 10 and 11 from the dataset and store as female
    female = family_count[10:12]
    # join the two dataframes
    family_count = pd.concat([female, male], ignore_index=True, axis = 0)
    # since we have unnecessary columns in between the ones we want, let's skip every other column
    columns = family_count.columns[2::2]
    # now let's drop every other column
    family_count.drop(columns, axis = 1, inplace = True)
    # for legibility, let's rename the index into named columns
    family_count = family_count.T.rename(columns = {0:'fem_total(count)', 1:'fem_with_kid(count)', 2:'male_total(count)', 3:'male_with_kid(count)'})
    # we still have a row that is full of the old title, let's remove that
    family_count = family_count.drop(family_count.index[0])

    # now we can remove null values and remove commas from our numbers
    family_count = family_count.fillna('0')
    family_count['fem_total(count)'] = family_count['fem_total(count)'].str.replace(',', '').astype(int)
    family_count['fem_with_kid(count)'] = family_count['fem_with_kid(count)'].str.replace(',', '').astype(int)
    family_count['male_total(count)'] = family_count['male_total(count)'].str.replace(',', '').astype(int)
    family_count['male_with_kid(count)'] = family_count['male_with_kid(count)'].str.replace(',', '').astype(int)

    # transform strings into floats
    family_count['fem_no_kid(count)'] = family_count['fem_total(count)'] - family_count['fem_with_kid(count)']
    family_count['male_no_kid(count)'] = family_count['male_total(count)'] - family_count['male_with_kid(count)']

    # grab the associated indices for male, female, and total
    tot = family_income[4:6]
    male = family_income[10:12]
    female = family_income[14:16]
    # bring those together
    family_income = pd.concat([tot, female, male], ignore_index=True, axis = 0)
    # for legibility, let's rename the index into named columns
    family_income = family_income.T.rename(columns = {0:'mar_with_kid', 1:'mar_no_kid', 2:'fem_with_kid(income)', 
                                                      3:'fem_no_kid(income)', 4:'male_with_kid(income)', 5:'male_no_kid(income)'})
    family_income = family_income.drop(family_income.index[0])

    # let's remove special characters from our strings
    family_income['mar_with_kid'] = family_income['mar_with_kid'].str.replace('+', '')
    family_income['male_no_kid(income)'] = family_income['male_no_kid(income)'].str.replace('+', '').replace('-', '')
    family_income['male_with_kid(income)'] = family_income['male_no_kid(income)'].str.replace('+', '').replace('-', '')

    # now let's remove all other special characters and fill some of our blank values with 0 to be dropped later
    family_income['mar_with_kid'] = family_income['mar_with_kid'].replace('', '0').str.replace(',', '').astype(int)
    family_income['mar_no_kid'] = family_income['mar_no_kid'].replace('', '0').str.replace(',', '').astype(int)
    family_income['fem_no_kid(income)'] = family_income['fem_no_kid(income)'].replace('', '0').str.replace(',', '').astype(int)
    family_income['fem_with_kid(income)'] = family_income['fem_with_kid(income)'].replace('', '0').str.replace(',', '').astype(int)
    family_income['male_no_kid(income)'] = family_income['male_no_kid(income)'].replace('', '0').str.replace(',', '').astype(int)
    family_income['male_with_kid(income)'] = family_income['male_with_kid(income)'].replace('', '0').str.replace(',', '').astype(int)

    # finally let's bring these two together
    family = pd.concat([family_income, family_count], axis = 1)
    
    # make the column names of col lowercase
    col.columns = col.columns.str.lower()
    col.rename(columns = {'areaname' :'msa'}, inplace=True)

In [4]:
family = family.reset_index()
family = family.rename(columns = {'index': 'msa'})

Abilene, TX MSA

In [11]:
family.msa[0][:-10]

'Abilene, TX Metro Area'

In [18]:
family.msa.index

RangeIndex(start=0, stop=392, step=1)

In [19]:
i = 0
for i in family.msa.index:
    family.msa[i] = family.msa[i][:-10]
    i +=1

/var/folders/sw/4nk3j1h92wb2t79pkgpn3gcw0000gp/T/ipykernel_81467/3256293206.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  family.msa[i] = family.msa[i][:-10]
/var/folders/sw/4nk3j1h92wb2t79pkgpn3gcw0000gp/T/ipykernel_81467/3256293206.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  family.msa[i] = family.msa[i][:-10]
/var/folders/sw/4nk3j1h92wb2t79pkgpn3gcw0000gp/T/ipykernel_81467/3256293206.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view

In [21]:
family.head()

,msa,mar_with_kid,mar_no_kid,fem_with_kid(income),fem_no_kid(income),male_with_kid(income),male_no_kid(income),fem_total(count),fem_with_kid(count),male_total(count),male_with_kid(count),fem_no_kid(count),male_no_kid(count)
0,"Abilene, TX Metro Area",94317,86894,29773,52525,63750,63750,16480,3675,11267,614,12805,10653
1,"Aguadilla-Isabela, PR Metro Area",37347,26726,10014,22547,28542,28542,0,0,0,0,0,0
2,"Akron, OH Metro Area",129867,96430,35631,65557,73787,73787,86856,16321,55642,3051,70535,52591
3,"Albany, GA Metro Area",95985,83303,22323,44698,56983,56983,21808,7019,10264,607,14789,9657
4,"Albany-Lebanon, OR Metro Area",89782,87816,45302,78237,50970,50970,10898,1821,8089,1018,9077,7071


In [8]:
col.head()

,case_id,state abv.,msa,family,housing,food,transportation,healthcare,other necessities,childcare,taxes,total,median_family_income
0,3172,TX,"Abilene, TX MSA",1p0c,8484.0,3014.58108,10162.77900,5389.47984,4166.38956,0.00000,4687.83576,35905.0668,66485.14729
1,3172,TX,"Abilene, TX MSA",1p1c,11460.0,4442.76048,10862.67996,8138.64552,5762.19684,6947.80152,4689.70308,52303.7868,66485.14729
2,3172,TX,"Abilene, TX MSA",1p2c,11460.0,6509.38548,11661.86724,10887.81084,6511.01664,11742.82104,4363.63296,63136.5348,66485.14729
3,3172,TX,"Abilene, TX MSA",1p3c,15420.0,8683.80756,12774.77520,13636.97640,8733.76020,13218.21240,5737.31256,78204.8436,66485.14729
4,3172,TX,"Abilene, TX MSA",1p4c,15420.0,10629.56832,13129.51440,16386.14160,9438.78588,13218.21240,5405.69532,83627.9184,66485.14729


In [7]:
len(family)

392

In [8]:
col.iloc[::10].head(30)

,case_id,state abv.,msa,family,housing,food,transportation,healthcare,other necessities,childcare,taxes,total,median_family_income
0,3172,TX,"Abilene, TX MSA",1p0c,8484.0,3014.58108,10162.77900,5389.47984,4166.38956,0.0,4687.83576,35905.0668,66485.14729
10,3173,OH,"Akron, OH MSA",1p0c,6792.0,3236.27340,9402.31344,4309.35024,3633.63900,0.0,4404.13476,31777.7112,77541.41057
20,3174,GA,"Albany, GA MSA",1p0c,7488.0,3182.99892,10238.85504,5149.47984,3866.52360,0.0,5757.92760,35683.7844,55237.98817
30,3175,OR,"Albany, OR MSA",1p0c,9528.0,3247.41468,11009.33424,4604.37996,4629.03660,0.0,7442.64480,40460.8092,68382.50732
40,3176,NY,"Albany-Schenectady-Troy, NY MSA",1p0c,9708.0,3679.23228,9954.05784,5668.53996,4850.72208,0.0,6768.18756,40628.7396,95476.30918
50,3177,NM,"Albuquerque, NM MSA",1p0c,7452.0,3221.06796,9490.17036,3476.37996,3867.27360,0.0,4569.05088,32075.9412,69364.11018
60,3178,LA,"Alexandria, LA MSA",1p0c,6480.0,2978.13132,10728.57132,5545.47984,3427.05396,0.0,5142.59580,34301.8332,58290.55052
70,3179,PA,"Allentown-Bethlehem-Easton, PA HUD Metro FMR Area",1p0c,9456.0,3454.48164,9336.75444,4680.69612,4677.97704,0.0,5798.74032,37404.6504,81129.97748
80,3180,NJ,"Warren County, NJ HUD Metro FMR Area",1p0c,11256.0,3683.16984,10869.48048,4672.53996,5413.05024,0.0,6285.60132,42179.8416,103354.14880
90,3181,PA,"Altoona, PA MSA",1p0c,6888.0,3185.16324,10100.36940,3508.53996,3649.90428,0.0,4813.14840,32145.1260,66065.98776


<hr style="border:2px solid black">

# **Prepare**

**Prepare Actions:**
* To better estimate family income based on family composition, use the estimates of family count by family type(Married, Single Male with kids, Single Male without kids, Single Female with kids, Single Female without kids) and the estimates of income by family type. e.g. (female_with_kids (income) * female_with_kids (count) + (male_with_kids (income) * male_with_kids (count)))/2
* Merge transformed census dataframe (family) on to the cost_of_living dataframe (col) on county.
* Adjust for inflation 2020 to 2022

In [ ]:
cost, dash = wrangle()
cost

,county,parents,children,housing,food,transportation,healthcare,other,childcare,taxes,total,median_income
0,baldwin_county_al,1,0,9390,4048,11695,6096,4869,0,7035,43133,68231
1,baldwin_county_al,1,1,12058,5966,12558,9202,6530,6498,7734,60546,40290
2,baldwin_county_al,1,2,12058,8740,14025,12309,7536,16726,10329,81723,40290
3,baldwin_county_al,1,3,16334,11660,14661,15415,10143,19874,14165,102253,40290
4,baldwin_county_al,1,4,16334,14273,15010,18522,11090,19874,14640,109742,40290
...,...,...,...,...,...,...,...,...,...,...,...,...
8239,natrona_county_wy,2,0,9638,6488,14377,15581,5843,0,6990,58917,94332
8240,natrona_county_wy,2,1,11351,8075,15391,19471,7039,10372,8267,79966,103027
8241,natrona_county_wy,2,2,11351,10227,16148,23361,7819,22325,9462,100693,103027
8242,natrona_county_wy,2,3,16242,12487,16629,27251,10410,26003,11935,120957,103027


<hr style="border:2px solid black">

# **Explore**
Plan --> Acquire --> Prepare --> **Explore** --> Model --> Deliver

<hr style="border:2px solid black">

# **Modeling**
Plan --> Acquire --> Prepare --> Explore --> **Model** --> Deliver


<hr style="border:2px solid black">

# **Comparing Models**


<hr style="border:2px solid black">

## **Best Model Evaluation on Test**

### <u>Modeling Summary:</u>

>* 

<hr style="border:2px solid black">

# **Conclusions**


### <u>Modeling:</u>

>**<u>:</u>**
>* 

### <u>Recommendations:</u>
>* 

### <u>Next Steps:</u>
>* 